In [31]:
import csv

from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import pandas as pd
import time

page = 'https://www.datacentermap.com/usa/'

def render_page(url):
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r
us_state_to_abbrev = {
    "alabama": "AL",
    "alaska": "AK",
    "arizona": "AZ",
    "arkansas": "AR",
    "california": "CA",
    "colorado": "CO",
    "connecticut": "CT",
    "delaware": "DE",
    "florida": "FL",
    "georgia": "GA",
    "hawaii": "HI",
    "idaho": "ID",
    "illinois": "IL",
    "indiana": "IN",
    "iowa": "IA",
    "kansas": "KS",
    "kentucky": "KY",
    "louisiana": "LA",
    "maine": "ME",
    "maryland": "MD",
    "massachusetts": "MA",
    "michigan": "MI",
    "minnesota": "MN",
    "mississippi": "MS",
    "missouri": "MO",
    "montana": "MT",
    "nebraska": "NE",
    "nevada": "NV",
    "new-hampshire": "NH",
    "new-jersey": "NJ",
    "new-mexico": "NM",
    "new-york": "NY",
    "north-carolina": "NC",
    "north-dakota": "ND",
    "ohio": "OH",
    "oklahoma": "OK",
    "oregon": "OR",
    "pennsylvania": "PA",
    "rhode-island": "RI",
    "south-carolina": "SC",
    "south-dakota": "SD",
    "tennessee": "TN",
    "texas": "TX",
    "utah": "UT",
    "vermont": "VT",
    "virginia": "VA",
    "washington": "WA",
    "west-virginia": "WV",
    "wisconsin": "WI",
    "wyoming": "WY",
    "district-of-columbia": "DC",
    "american-samoa": "AS",
    "guam": "GU",
    "northern-mariana-islands": "MP",
    "puerto-rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

def scraper(page, states):
    output = pd.DataFrame()
    datacenter_list = []
    for s in states:
        url = str(page) + s
        print(url)
        r = render_page(url)

        soup = BS(r, "html.parser")
        table = soup.findAll("table")[0]

        for el in soup.findAll('tr'):
            data = el.findAll('td')
            if not data:
                continue
            city_name = data[0].findAll('a')[0].text
            number = data[1].text

            one_problem = [s, city_name, number]
            datacenter_list.append(one_problem)
        file=us_state_to_abbrev[s]+'_us-states-datacenters.csv'
        print(file)
        print(datacenter_list)
        with open(file, 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerows(datacenter_list)


In [36]:
scraper(page, ["illinois"])

https://www.datacentermap.com/usa/illinois
IL_us-states-datacenters.csv
[['illinois', 'Chicago', '101'], ['illinois', 'Bloomington', '2'], ['illinois', 'Peoria', '1'], ['illinois', 'Champaign', '1'], ['illinois', 'Aurora', '1'], ['illinois', 'Rockford', '1'], ['illinois', 'Rantoul', '1']]


In [37]:
file='IL_us-states-datacenters.csv'
with open(file, 'r') as f:
    statename=file.split('_')[0]
    cityname=[]
    for row in f:
        second_data = cityname.append(row.split(',')[1].replace(' ', '-'))
        
print(cityname)
print(statename)

['"Chicago"', '"Bloomington"', '"Peoria"', '"Champaign"', '"Aurora"', '"Rockford"', '"Rantoul"']
IL


In [1]:

from selenium import webdriver
from selenium.webdriver.common.by import By


def get_station_keyword(state, city_name):
    driver = webdriver.Chrome()
    driver.get("https://www.wunderground.com/history/daily/us/" + state + "/" + city_name)
    time.sleep(3)
    driver.find_element(By.XPATH, "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[1]/div[1]/div/lib-date-selector/div/input").click()
    return driver.current_url.split('/')[8]
    


In [2]:
def row_delete(df_check):
    tt=df_check['Time'].tolist()
    delrow=[]
    for i in range (len(tt)-1):
        curr=int(tt[i].split(':')[0])
        next=int(tt[i+1].split(':')[0])
        if curr==next:
            delrow.append((i))
    return (delrow)
            

In [3]:
import datetime
st = datetime.datetime.now()
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import pandas as pd
import time


def render_page(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r


def scraper(page, locations, dates,city):
    df = pd.DataFrame()


    for l in locations:

        for d in dates:
            print(d)
            url = str(str(page) + l + "/date/" + str(d))
#             print(url)
            r = render_page(url)

            soup = BS(r, "html.parser")
            container = soup.find('lib-city-history-observation')
            check = container.find('tbody')

            daily_data = []
            data = []

            for c in check.find_all('tr', class_='ng-star-inserted'):
                for i in c.find_all('td', class_='ng-star-inserted'):
                    trial = i.text
                    trial = trial.strip('  ')
                    data.append(trial)

#             print(data)
#             print(len(data))
#             time.sleep(3)
#             time.sleep(1)
            for i in range(0, len(data), 10):
#                 hourly_data=[dates, data[i:i + 10]]
                hourly_data = data[i:i + 10]
                daily_data.append(hourly_data)
#             print(daily_data)
            s=daily_data
            df1 = pd.DataFrame(s,columns =['Time', 'Temperature', 'DewPoint', 'Humidity', 'Wind', 'WindSpeed', 'WindGust', 'Pessure','Precip', 'Condition'])
#             df1 = pd.DataFrame(s) 
#             pd.concat([df, df1])
#             df.to_csv(l+'.csv')
#             df_check = pd.read_csv(l+'.csv')
#             print(df )
            index=row_delete(df1)
#             print(index)
            df_final=df1.drop(index)
#             print(df_final)
#             print(df_final.shape[0])
#             df_final.insert(0, "Date", [next(fruits) for fruit in range(len(df))], True)
            df_final.insert(0, "Date",0)

        

            list_ = [d]
            ser = list_ * int(len(df_final)/len(list_))
            df_final['Date'] = ser + list_[:len(df_final)-len(ser)]
            
            df_final["timestamp"] = pd.to_datetime(df_final["Date"] + " " + df_final["Time"], format="%Y-%m-%d %I:%M %p")
            df_final = df_final.sort_values(by='timestamp')

            
#             print(df_final)
            df = pd.concat([df, df_final], ignore_index=True)
#             df.append(df_final, ignore_index=True)
#             pd.concat([df, df_final])
#             print(d)

            
            
#             df_final.to_csv(city+'.csv')
        
        df.to_csv(city+'.csv',index=False)
        
        print('Scraper done!')
            

#             return df_final
# dates = ['2024-1-5']

# locations = [(get_station_keyword(state, city))]
# print(locations)
# page = 'https://www.wunderground.com/history/daily/'
# df_output = scraper(page, locations, dates,city)
# et = datetime.datetime.now()
# elapsed_time = et - st
# print('Execution time:', elapsed_time, 'seconds')

In [4]:
from datetime import date
city=["Santa-Clara","Philadelphia" ]
states=["CA","PA"]

start_dt = date(2023, 1, 1)
end_dt = date(2023, 12, 31)

from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)


dates=[]
for dt in daterange(start_dt, end_dt):
    dates.append((dt.strftime("%Y-%m-%d")))

for cityname in city:
    
    st = datetime.datetime.now()
    locations = [(get_station_keyword(states[city.index(cityname)], cityname))]
    print(locations)
    page = 'https://www.wunderground.com/history/daily/'
    df_output = scraper(page, locations, dates,cityname)
    et = datetime.datetime.now()
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')

['KSJC']
2023-01-01
2023-01-02
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-07
2023-01-08
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-14
2023-01-15
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-21
2023-01-22
2023-01-23
2023-01-24
2023-01-25
2023-01-26
2023-01-27
2023-01-28
2023-01-29
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-04
2023-02-05
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-11
2023-02-12
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-18
2023-02-19
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-25
2023-02-26
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-11
2023-03-12
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
2023-03-19
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-25
2023-03-26
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2